In [10]:
import pandas as pd
import numpy as np
from sklearn import metrics, linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMRegressor

lgb = pd.read_csv('../lgCV_2505.csv.gz', compression='gzip')
rnn = pd.read_csv('../rnnCV_2805.csv.gz', compression='gzip')
mlp = pd.read_csv('../mlpCV_2505.csv.gz', compression='gzip')
truth = pd.read_csv('../../train.csv.zip', compression='zip')
test =  pd.read_csv('../../test.csv.zip', compression='zip')
test['deal_probability']=float('NAN') 
truth = pd.concat([truth,test[truth.columns]],axis=0)

lgb.rename(columns={'deal_probability': 'lgb_preds' }, inplace=True)
mlp.rename(columns={'deal_probability': 'mlp_preds' }, inplace=True)
preds_df = lgb.merge(rnn, on='item_id').merge(mlp, on='item_id').merge(truth, on='item_id',how='left')
preds_df['d']=pd.to_datetime(train_df['activation_date']).dt.day
train_df = preds_df[~preds_df['deal_probability'].isnull()]
test_df = preds_df[preds_df['deal_probability'].isnull()]

train_df['error_lgb'] = np.abs(train_df['deal_probability']-train_df['lgb_preds'])
train_df['error_rnn'] = np.abs(train_df['deal_probability']-train_df['rnn_preds'])
train_df['error_mlp'] = np.abs(train_df['deal_probability']-train_df['mlp_preds'])


/home/rosenflanzt/anaconda3/envs/kaggle_avito/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/rosenflanzt/anaconda3/envs/kaggle_avito/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/rosenflanzt/anaconda3/envs/kaggle_avito/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [11]:
def rmse(x):
    return np.sqrt((x*x).mean())
train_df.groupby(['category_name', 'd']).agg({'error_lgb':rmse, 'error_rnn':rmse, 'error_mlp':rmse, 'deal_probability':['mean','count']  })

error_lgb error_rnn error_mlp deal_probability  \
                                rmse      rmse      rmse             mean   
category_name         d                                                     
Автомобили            2.0   0.266110  0.164966  0.246483         0.000000   
                      3.0   0.355356  0.493162  0.472587         0.737600   
                      7.0   0.586436  0.638739  0.572114         0.737600   
                      15.0  0.260385  0.269430  0.272113         0.274693   
                      16.0  0.257916  0.267588  0.269208         0.271391   
                      17.0  0.257890  0.268198  0.270396         0.287975   
                      18.0  0.258890  0.269084  0.272203         0.291925   
                      19.0  0.264161  0.270797  0.274375         0.280506   
                      20.0  0.255301  0.262668  0.267676         0.267863   
                      21.0  0.257282  0.266993  0.269804         0.278428   
                      22.0  0.262840  0.267218  0.271800         0.284962   
                      23.0  0.259892  0.265166  0.268460         0.275441   
                      24.0  0.253249  0.261149  0.265659         0.278178   
                      25.0  0.260639  0.270190  0.274230         0.283222   
                      26.0  0.264150  0.273219  0.275607         0.276313   
                      27.0  0.257181  0.264699  0.268712         0.265818   
                      28.0  0.261628  0.269780  0.274044         0.283586   
                      29.0  0.395299  0.415104  0.411339         0.737600   
                      30.0  0.068239  0.175798  0.201685         0.000000   
Аквариум              15.0  0.259401  0.265969  0.266229         0.196349   
                      16.0  0.252107  0.252579  0.259317         0.176588   
                      17.0  0.252023  0.256701  0.256520         0.166565   
                      18.0  0.245211  0.253190  0.253519         0.160599   
                      19.0  0.257637  0.257806  0.261467         0.183236   
                      20.0  0.263622  0.262262  0.262891         0.222212   
                      21.0  0.246634  0.243903  0.247130         0.180761   
                      22.0  0.260989  0.264514  0.261406         0.179673   
                      23.0  0.257567  0.265414  0.265228         0.199302   
                      24.0  0.254697  0.248628  0.254759         0.175738   
                      25.0  0.263843  0.261645  0.260871         0.168428   
...                              ...       ...       ...              ...   
Товары для компьютера 29.0  0.566427  0.588330  0.557108         0.767860   
Фототехника           15.0  0.241273  0.245519  0.245700         0.124923   
                      16.0  0.240291  0.239728  0.243370         0.134456   
                      17.0  0.228338  0.230686  0.225602         0.119603   
                      18.0  0.228868  0.234341  0.233109         0.120280   
                      19.0  0.256404  0.258856  0.261975         0.157625   
                      20.0  0.238404  0.236740  0.241447         0.127759   
                      21.0  0.223504  0.227807  0.229438         0.120571   
                      22.0  0.232071  0.228101  0.228457         0.120471   
                      23.0  0.246044  0.250100  0.254477         0.138089   
                      24.0  0.249204  0.255379  0.254963         0.136845   
                      25.0  0.231475  0.235431  0.235808         0.118937   
                      26.0  0.256541  0.260016  0.258792         0.147587   
                      27.0  0.246575  0.247266  0.248654         0.135945   
                      28.0  0.248935  0.256644  0.252142         0.144408   
Часы и украшения      15.0  0.157698  0.158316  0.157325         0.054052   
                      16.0  0.159345  0.160036  0.158836         0.053570   
                      17.0  0.157918  0.157364  0.156271         0.054126   
                      1

In [12]:
train_df.groupby(['d']).agg({'error_lgb':rmse, 'error_rnn':rmse, 'error_mlp':rmse, 'deal_probability':['mean','count']  })

error_lgb error_rnn error_mlp deal_probability        
          rmse      rmse      rmse             mean   count
d                                                          
1.0   0.720465  0.739661  0.730245         0.803230       3
2.0   0.385525  0.370808  0.368743         0.255953       3
3.0   0.278819  0.366602  0.362085         0.368800       2
7.0   0.586436  0.638739  0.572114         0.737600       1
15.0  0.216951  0.218202  0.219087         0.140073  108615
16.0  0.218476  0.219932  0.220794         0.141151  106168
17.0  0.216698  0.218095  0.219067         0.141158   98773
18.0  0.218745  0.220250  0.221321         0.140338   97554
19.0  0.220590  0.221615  0.222734         0.137609  114416
20.0  0.213641  0.215195  0.216364         0.137785  115190
21.0  0.215591  0.216820  0.217678         0.138985  110535
22.0  0.215791  0.215082  0.216266         0.138858  109813
23.0  0.215622  0.215369  0.216042         0.137318  106544
24.0  0.215784  0.217299  0.217920         0.139919   97351
25.0  0.219861  0.220957  0.221947         0.141355   97104
26.0  0.219209  0.220272  0.221043         0.138612  113513
27.0  0.213717  0.214343  0.216077         0.136219  114863
28.0  0.214389  0.214730  0.216303         0.138975  112885
29.0  0.499136  0.509499  0.495236         0.696854      87
30.0  0.075291  0.115405  0.134639         0.000000       3
31.0  0.000000  0.025960  0.026975         0.000000       1